In [2]:
# -*- coding: utf-8 -*-
# Ce code a Ã©tÃ© produit par le Pr Ricco R. de l'univ. de Lyon 2
#UtilisÃ© et modifiÃ© par Kenza TAAMMA et Bertille TEMPLE (crÃ©ation des projections 2D et 3D, de la moyenne des attributs/label ainsi que de la classification d'un nouvel utilisateur)

Modification du dossier par defaut

In [3]:
import os
os.chdir("./")#votre dossier

Importation des donnees

In [4]:
import pandas

In [5]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn import cluster

In [6]:
import pprint

In [7]:
import os, sys, types
if "import_ipynb" not in sys.modules:
    !pip install import-ipynb
import import_ipynb
sys.path.append(os.getcwd()+"\service")
from service import rethinkDBservice as rethinkDBservice

In [8]:
users = list(rethinkDBservice.getUsersCursors())
table = []
# cmp = 0

KeyboardInterrupt: 

In [ ]:
for user in users:
    table.append([
        user["accountAge(days)"],
        user["agressivite"],
        user["avg_hashtag"],
        user["avg_url"],
        user["mediumLength"],
        user["rationFollowersFriends"],
        user["tweet_per_day"],
        user["verified"],
        user["visibility"]
    ])

In [ ]:
matriceDonnees = np.array(table)

In [ ]:
dataFrameDonnees = pandas.DataFrame(data=matriceDonnees, columns=["accountAge", "agressivite", "avg_hashtag", "avg_url", "mediumLength", "rationFollowersFriends", "tweet_per_day", "verified", "visibility"])

Dimension des donnees

In [ ]:
print(dataFrameDonnees.shape)

 premiÃ¨res lignes des donnÃ©es

In [ ]:
print(dataFrameDonnees.iloc[0:6,:])

Statistiques descriptives

In [ ]:
print(dataFrameDonnees.describe())

#graphique avec croisement deux Ã  deux<br>
from pandas.plotting import scatter_matrix<br>
pandas.plotting.scatter_matrix(dataFrameDonnees,figsize=(11,11)) 

Centrage reduction des donnees

In [ ]:
from sklearn import preprocessing
dataFrameDonnees_cr = preprocessing.scale(dataFrameDonnees)

Librairie pour evaluation des partitions

In [ ]:
from sklearn import metrics

Utilisation de la metrique "silhouette"<br>
Faire varier le nombre de clusters de 2 Ã  9<br>
creation d'un index alÃ©atoire de 10 000 user pour l'affichage sinon trop long

In [ ]:
idx = np.random.randint(len(matriceDonnees), size=10000)
dataFrameDonnees_affichage = dataFrameDonnees_cr[idx]
res = np.arange(9,dtype="double")
for k in np.arange(9):
    km = cluster.KMeans(n_clusters=k+2)
    km.fit(dataFrameDonnees_affichage)
    res[k] = metrics.silhouette_score(dataFrameDonnees_affichage,km.labels_)

Graphique

In [ ]:
import matplotlib.pyplot as plt
plt.title("Silhouette")
plt.xlabel("# of clusters")
plt.plot(np.arange(2,11,1),res)
plt.show()

=============== Fonction deroulant l'etude du clustering KMeans sur notre dataset =============== #<br>
=============== En fonction du nombre de cluster passé en argument =============== #

In [ ]:
def etudeClusteringKmeans(nb_cluster):
    #k-means sur les donnÃ©es centrÃ©es et rÃ©duites
    kmeans = cluster.KMeans(n_clusters=nb_cluster)
    kmeans.fit(dataFrameDonnees_cr)
    # print(kmeans.inertia_)

    #index triÃ©s des groupes
    idk = np.argsort(kmeans.labels_)

    #affichage des observations et leurs groupes
    print(pandas.DataFrame(dataFrameDonnees.index[idk],kmeans.labels_[idk]))

    #distances aux centres de classes des observations
    print(kmeans.transform(dataFrameDonnees_cr))

    #moyenne par variable
    m = dataFrameDonnees.mean()

    #data.frame conditionnellement aux groupes
    gb = dataFrameDonnees.groupby(kmeans.labels_)

    #ACP pour affichage
    from sklearn.decomposition import PCA
    acp = PCA(n_components=2).fit_transform(dataFrameDonnees_cr)
    # from AcpTweet import CP2
    # acp = CP2
    # print("#### ACP #####")
    # print (acp)

    # ------- Projection 2D ------- #

    # discrÃ©tise l'Ã©chelle de couleur viridis pour l'affichage
    cmap = plt.get_cmap("viridis", nb_cluster)

    # crÃ©ation d'un index alÃ©atoire de 10 000 user pour l'affichage
    idx = np.random.randint(len(matriceDonnees), size=10000)

    # crÃ©ation du dataframe avec les CP et les labels correspondants par concatÃ©nation
    acp_df = pandas.DataFrame(data=acp[idx,:2], columns=["CP1", "CP2"])
    labels = pandas.DataFrame(data=kmeans.labels_[idx], columns=["label"])
    labeled_acp = pandas.concat([acp_df, labels], axis=1)
    print("----------- labeled acp -----------")
    print(labeled_acp)

    # crÃ©ation des groupes pour l'affichage
    clusters = labeled_acp.groupby("label")
    # projection dans le plan factoriel
    # avec un code couleur selon le groupe
    # plt.scatter(acp[:,0],acp[:,1], c=kmeans.labels_, cmap='viridis', label = name) #sans le groupby et l'affichage des legendes
    for name, group in clusters :
        plt.scatter(group["CP1"],group["CP2"], color=cmap(int(name)), label = name) 
    plt.xlabel("Composante Principale 1")
    plt.ylabel("Composante Principale 2")
    plt.title("Clustering KMeans "+ str(nb_cluster) + " clusters - projection 2D")
    plt.legend() 

    # ------- Projection 3D ------- #

    # ACP pour affichage 3D
    acp = PCA(n_components=3).fit_transform(dataFrameDonnees_cr)
    # from AcpTweet import CP3
    # acp = CP3
    # crÃ©ation du dataframe concatÃ©nÃ© aux labels
    acp_df = pandas.DataFrame(data=acp[idx,:3], columns=["CP1", "CP2", "CP3"])
    labels = pandas.DataFrame(data=kmeans.labels_[idx], columns=["label"])
    labeled_acp = pandas.concat([acp_df, labels], axis=1)
    # crÃ©ation des groupes pour l'affichage
    clusters = labeled_acp.groupby("label")

    # projection 3D
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111, projection='3d')
    # ax.scatter(acp[:,0],acp[:,1],acp[:,2], c=kmeans.labels_, cmap='viridis', edgecolor='k', s=40, alpha = 0.5) #sans le groupby et l'affichage des legendes
    for name, group in clusters :
        ax.scatter(group["CP1"],group["CP2"], group["CP3"], color=cmap(int(name)), edgecolor='k', s=40, alpha = 0.5, label = name)
    ax.view_init(60,35)
    ax.set_title("Clustering KMeans "+ str(nb_cluster) + " clusters - projection 3D")
    ax.set_xlabel("Composante principale 1")
    ax.set_ylabel("Composante principale 2")
    ax.set_zlabel("Composante principale 3")
    ax.dist = 10
    ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], c='black', s=200, alpha=0.5, label = 'Centroid')
    plt.legend() 
    plt.autoscale(enable=True, axis='x', tight=True)    

    # autre projection 3D - rotation pour une meilleure visualisation
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111, projection='3d')
    # ax.scatter(acp[:,0],acp[:,1],acp[:,2], c=kmeans.labels_, cmap='viridis', edgecolor='k', s=40, alpha = 0.5) #sans le groupby et l'affichage des legendes
    for name, group in clusters :
        ax.scatter(group["CP1"],group["CP2"], group["CP3"], color=cmap(int(name)), edgecolor='k', s=40, alpha = 0.5, label = name)
    ax.set_title("Clustering KMeans "+ str(nb_cluster) + " clusters - projection 3D")
    ax.set_xlabel("Composante principale 1")
    ax.set_ylabel("Composante principale 2")
    ax.set_zlabel("Composante principale 3")
    ax.dist = 10
    ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], c='black', s=200, alpha=0.5, label = 'Centroid')
    plt.legend() 
    plt.autoscale(enable=True, axis='x', tight=True)    

    # affichage des 3 graphiques (2D et 3D)
    plt.show()
    pred = kmeans.predict(dataFrameDonnees_cr)
    from sklearn import metrics
    score = metrics.rand_score(pred, kmeans.labels_)
    print("---------------- Performance prÃ©diction ----------------")
    print(score)
    print("---------------- Etude classes ----------------\n Voir fichier Etude_classes.txt")
    # 30 users pour l'affichage
    dataFrameDonnees_affichage = dataFrameDonnees.loc[0:30,:]
    labels = pandas.DataFrame(data=kmeans.labels_, columns=["label"])
    labeled_dataFrame = pandas.concat([dataFrameDonnees_affichage, labels[0:31]], axis=1)
    # pour le calcul des moyennes des attributs par label
    labeled_dataFrame_groupby = pandas.concat([dataFrameDonnees_affichage, labels], axis=1)
    labeled_dataFrame_groupby = labeled_dataFrame_groupby.groupby(['label']).mean()
    
    # pour afficher toutes les colonnes
    pandas.set_option('display.max_columns', None)
    
    print(labeled_dataFrame, file = sourceFile)
    print('Moyennes : ', file = sourceFile)
    print(labeled_dataFrame_groupby, file = sourceFile)
    # affiche en plus dans la console
    print('Moyennes : ')
    print(labeled_dataFrame_groupby, file = sourceFile)
    pandas.reset_option("max_columns")

=============== Etude clustering KMeans sur notre dataset =============== #

In [1]:
sourceFile = open('Etude_classes.txt', 'w')
# etude en fonction du nombre de clusters
# print(" ----------- 2 clusters -----------")
# etudeClusteringKmeans(2)
# print(" ----------- 3 clusters -----------")
# etudeClusteringKmeans(3)
print(" ----------- 4 clusters -----------")
etudeClusteringKmeans(4)
sourceFile.close()

 ----------- 4 clusters -----------


NameError: name 'etudeClusteringKmeans' is not defined

=============== Fonction pour assigner un cluster Ã  une nouvelle valeure =============== #

In [ ]:
def classerNewUserKMeans(user, nb_cluster):
    user = np.array(user)
    kmeans = cluster.KMeans(n_clusters=nb_cluster)
    kmeans.fit(dataFrameDonnees_cr)
    # baricentres des n clusters
    # print(kmeans.cluster_centers_)

    #predict sinon
    pred = kmeans.predict(user)

    # discrÃ©tise l'Ã©chelle de couleur viridis pour l'affichage
    cmap = plt.get_cmap("viridis", nb_cluster)

    # crÃ©ation d'un index alÃ©atoire de 10 000 user pour l'affichage
    idx = np.random.randint(len(matriceDonnees), size=10000)

    # projection 3D pour visualiser les labels

    # ACP pour affichage 3D
    acp = PCA(n_components=3).fit_transform(dataFrameDonnees_cr)
    # crÃ©ation du dataframe concatÃ©nÃ© aux labels
    acp_df = pandas.DataFrame(data=acp[idx,:3], columns=["CP1", "CP2", "CP3"])
    labels = pandas.DataFrame(data=kmeans.labels_[idx], columns=["label"])
    labeled_acp = pandas.concat([acp_df, labels], axis=1)
    # crÃ©ation des groupes pour l'affichage
    clusters = labeled_acp.groupby("label")

    # projection
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111, projection='3d')
    # ax.scatter(acp[:,0],acp[:,1],acp[:,2], c=kmeans.labels_, cmap='viridis', edgecolor='k', s=40, alpha = 0.5) #sans le groupby et l'affichage des legendes
    for name, group in clusters :
        ax.scatter(group["CP1"],group["CP2"], group["CP3"], color=cmap(int(name)), edgecolor='k', s=40, alpha = 0.5, label = name)
    ax.view_init(60,35)
    ax.set_title("Clustering KMeans "+ str(nb_cluster) + " clusters - projection 3D")
    ax.set_xlabel("Composante principale 1")
    ax.set_ylabel("Composante principale 2")
    ax.set_zlabel("Composante principale 3")
    ax.dist = 10
    ax.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2], c='black', s=200, alpha=0.5, label = 'Centroid')
    plt.legend() 
    plt.autoscale(enable=True, axis='x', tight=True)    
    for i in range(user.shape[0]):
        if user[i,7] == True:
            user[i,7] = 1
        else :
            user[i,7] = 0
    baricentres = kmeans.cluster_centers_
    distances=np.zeros((1,nb_cluster)) 
    for i in range(user.shape[0]):
        for n in range(nb_cluster):
            distances[0,n]=sum((user[i,:]-baricentres[n,:])*(user[i,:]-baricentres[n,:]))

    # affichage du rÃ©sultat de la classe en fonction de la distance au baricentre 
    # et de la classe retournÃ© par la fonction sklearn.kmeans.predict
    print("--------- Distances aux baricentres ---------")
    print(distances)
    id_proche=np.argmin(distances) 
    print("class = "+str(id_proche)) 
    print("prÃ©diction :")
    print(pred)
    print('FIN')
    plt.show()

=============== Utilisateur Ã  classifier =============== #

In [ ]:
user = [[12, 2, 1, 4, 80, 0.5, 2, True, 0.0821]]
# print(user[9])
classerNewUserKMeans(user, 4)